In [ ]:
import torch
import tiktoken

In [ ]:
with open ("Amontillado.txt", "r") as f:
    raw_text = f.read()

raw_text[:50]

'The thousand injuries of Fortunato I had borne as '

In [4]:
tokenizer = tiktoken.get_encoding("gpt2")


In [5]:
enc_text = tokenizer.encode(raw_text)

In [6]:
print(enc_text[:20])

[464, 7319, 6821, 286, 376, 1922, 5549, 314, 550, 28068, 355, 314, 1266, 714, 26, 475, 618, 339, 44716, 2402]


In [7]:
print(tokenizer.decode(enc_text[:20]))

The thousand injuries of Fortunato I had borne as I best could; but when he ventured upon


In [ ]:
for i in range(1,10):
    print("Input:", tokenizer.decode(enc_text[:i]), "Target", tokenizer.decode([enc_text[i]]))

TypeError: argument 'tokens': 'int' object cannot be converted to 'Sequence'

In [ ]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasedef create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloadertV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


In [21]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [26]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)


Inputs:
 tensor([[  464,  7319,  6821,   286],
        [  376,  1922,  5549,   314],
        [  550, 28068,   355,   314],
        [ 1266,   714,    26,   475],
        [  618,   339, 44716,  2402],
        [13277,    11,   314, 19982],
        [15827,    13,   921,    11],
        [  508,   523,   880,   760]])

Targets:
 tensor([[ 7319,  6821,   286,   376],
        [ 1922,  5549,   314,   550],
        [28068,   355,   314,  1266],
        [  714,    26,   475,   618],
        [  339, 44716,  2402, 13277],
        [   11,   314, 19982, 15827],
        [   13,   921,    11,   508],
        [  523,   880,   760,   262]])
